# Task 1 : MongoDB Data Model

##### Based on the two data sets provided i.e. hotspot_historic.csv and climate_historic.csv, we have designed a suitable data model to support the efficient storing and querying of the two data sets in MongoDB. We know that the climate and fire data will be gathered in large quantities over the years whcih makes us think about a model which is easy to scale and yet have a distinguised event entities among different collections in database. We will be the reference model. As we observe that both "date" fields of the dataset are similar and can be compared, we have decided to join them based on that field when querying. A similar date for both datasets would imply that on that day with that particular climate report, there is a fire in that area.

##### The reason is such that although the datasets are relatively small, but given that MongoDB has a limit of 16MB of data size, other models such as embedding model might not be able to scale to such an large amount of data. Besides, by referencing, we save up data space as well when we avoid repetition of climate data when we embed only one hotspot object into the climate data. Our data model adresses the following: high scalability and  keep the information free from redundancies.

##### However, the only downside of our data model is that it increases the number of join query operations. We believe that analysis and query in a climate and hotspot data are performed only based on a small section of fields. This is similar to the array storage structure, which can significantly reduce data scanning and reduce the impact on query performance. Mongo DB is one of those databases with the table elimination feature that can greatly reduce the number of join operations.

## Schema Creation 

##### We create the schema to ease with queries and also gives us an idea of how our model would look like. 

In [1]:
# schema to validate the climate collection.
{
        "station": {
            "description": "Constant station ID"
        },
        "date": {
            "description": "Date in datetime format"
        },
        "air_temperature_celcius": {
            "description": "Temperature in int format"
        },
        "relative_humidity": {
            "description": "Humidity in int format"
        },
        "windspeed_knots": {
            "description": "Windspeed in int format"
        },
        "max_wind_speed": {
            "description": "Windspeed in int format"
        },
        "precipitation": {
            "description": "Precipitation in int format"
        },
        "precipitation_type": {
            "description": "Type in str format"
        },
        "GHI_w/m2": {
            "description": "int format"
        }
}


{'station': {'description': 'Constant station ID'},
 'date': {'description': 'Date in datetime format'},
 'air_temperature_celcius': {'description': 'Temperature in int format'},
 'relative_humidity': {'description': 'Humidity in int format'},
 'windspeed_knots': {'description': 'Windspeed in int format'},
 'max_wind_speed': {'description': 'Windspeed in int format'},
 'precipitation': {'description': 'Precipitation in int format'},
 'precipitation_type': {'description': 'Type in str format'},
 'GHI_w/m2': {'description': 'int format'}}

In [2]:
# schema to validate the hotspot collection.
{
        "latitude": {
            "description": "Latitude in int format"
        },
        "longitude": {
            "description": "Longitude in int format"
        },
        "datetime": {
            "description": "Date and time in datetime format"
        },
        "confidence": {
            "description": "Confidence in int format"
        },
        "date": {
            "description": "Date in date format"
        },
        "surface_temperature_celcius": {
            "description": "Temperature in int format"
        }
}

{'latitude': {'description': 'Latitude in int format'},
 'longitude': {'description': 'Longitude in int format'},
 'datetime': {'description': 'Date and time in datetime format'},
 'confidence': {'description': 'Confidence in int format'},
 'date': {'description': 'Date in date format'},
 'surface_temperature_celcius': {'description': 'Temperature in int format'}}

##### An example of the MongoDB model for climate data and hotspots data can be seen below, respectively.

{<br>
 '_id': ObjectId('628f22a0500536bc72322b80'), <br>
 'station': 948700, <br>
 'date': datetime.datetime(2020, 12, 31, 0, 0), <br>
 'air_temperature_celcius': 19, <br>
 'relative_humidity': 56.8, <br>
 'windspeed_knots': 7.9, <br>
 'max_wind_speed': 11.1, <br>
 'precipitation ': 0.0, <br>
 'GHI_w/m2': 154, <br>
 'precipitation_type': 'I' <br> }

{<br>'_id': ObjectId('628961b952e0577c296e7dc4'), <br>
 'latitude': -37.966,<br>
 'longitude': 145.05100000000002,<br>
 'datetime': datetime.datetime(2021, 12, 27, 4, 16, 51),<br>
 'confidence': 78,<br>
 'date': datetime.datetime(2021, 12, 27, 0, 0),<br>
 'surface_temperature_celcius': 68<br>}

# Task 2. Querying MongoDB using PyMongo


## 1. Reading data from csv to MongoDB.

In [3]:
# import the following libraries to perform this task.
from pymongo import MongoClient # access mongoDb using python.
import datetime 
import pandas as pd
from datetime import datetime
import dateutil.parser
import pymongo
from pprint import pprint

In [4]:
# instantiating mongo client.
client = pymongo.MongoClient()

# instantiating the database
db = client.fit3182_db

In [5]:
# instantiating the collections in the created database.
climate_collection = db.climate # collection named "climate"
hotspot_collection = db.hotspot # collection names "hotspot"

In [6]:
# read the csv file into respective variables.
climate = pd.read_csv("climate_historic.csv")
hotspot = pd.read_csv("hotspot_historic.csv")

In [7]:
# emptying the collections to prevent repetitive data
climate_collection.drop()
hotspot_collection.drop()

In [8]:
# transform the climate date to datetime format. We also preprocess the precipitation data into two columns, precipitation value and precipitation type.
climate["date"] = pd.to_datetime(climate["date"], format="%d/%m/%y")
climate["precipitation_type"] = climate["precipitation "].str[-1]
climate["precipitation "] = climate["precipitation "].str.slice(0,5) # precipitation type is at the 5th string index
climate["precipitation "] = pd.to_numeric(climate["precipitation "])

In [9]:
# transform the hotspot date and hotspot time to datetime format.
hotspot["datetime"] = pd.to_datetime(hotspot["datetime"], format="%Y-%m-%dT%H:%M:%S")
hotspot["date"] = pd.to_datetime(hotspot["date"], format="%d/%m/%Y")

In [10]:
# instantiate the csv as dictionaries (json format)
climate_dict = climate.to_dict(orient = "records")
hotspot_dict = hotspot.to_dict(orient = "records")

In [11]:
climate_collection.insert_many(climate_dict)

In [12]:
hotspot_collection.insert_many(hotspot_dict)

In [13]:
#finds one object from the climate collection
climate_collection.find_one()

{'_id': ObjectId('6290c48ca64bd561823ffcc7'),
 'station': 948700,
 'date': datetime.datetime(2020, 12, 31, 0, 0),
 'air_temperature_celcius': 19,
 'relative_humidity': 56.8,
 'windspeed_knots': 7.9,
 'max_wind_speed': 11.1,
 'precipitation ': 0.0,
 'GHI_w/m2': 154,
 'precipitation_type': 'I'}

In [14]:
hotspot_collection.find_one()

{'_id': ObjectId('6290c48da64bd561823ffe35'),
 'latitude': -37.966,
 'longitude': 145.05100000000002,
 'datetime': datetime.datetime(2021, 12, 27, 4, 16, 51),
 'confidence': 78,
 'date': datetime.datetime(2021, 12, 27, 0, 0),
 'surface_temperature_celcius': 68}

## 2. PyMongo Queries

##### a. Find climate data on 12th December 2021.


In [15]:
results = climate_collection.find({'date': datetime(2021, 12, 12)}) #matches the desired date 
for x in results:
    pprint(x)

{'GHI_w/m2': 156,
 '_id': ObjectId('6290c48ca64bd561823ffe20'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2021, 12, 12, 0, 0),
 'max_wind_speed': 12.0,
 'precipitation ': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


##### b. Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C.


In [16]:
# find the surface_temperature_celcius using gte and lte, and then show required fields, sort by temperature for easy viewing
results = hotspot_collection.find({"surface_temperature_celcius":{"$gte" : 65,"$lte":100}}, 
                               {"latitude": 1, "longitude":1,"confidence": 1,"surface_temperature_celcius":1
                                }).sort([("surface_temperature_celcius",1)])


for x in results:
    pprint(x)

{'_id': ObjectId('6290c48da64bd561823ffe6a'),
 'confidence': 100,
 'latitude': -37.641999999999996,
 'longitude': 149.263,
 'surface_temperature_celcius': 65}
{'_id': ObjectId('6290c48da64bd561823ffe8d'),
 'confidence': 87,
 'latitude': -37.861999999999995,
 'longitude': 144.175,
 'surface_temperature_celcius': 65}
{'_id': ObjectId('6290c48da64bd561823ffeac'),
 'confidence': 90,
 'latitude': -37.330999999999996,
 'longitude': 143.122,
 'surface_temperature_celcius': 65}
{'_id': ObjectId('6290c48da64bd561823ffeeb'),
 'confidence': 88,
 'latitude': -37.46,
 'longitude': 148.102,
 'surface_temperature_celcius': 65}
{'_id': ObjectId('6290c48da64bd561823fff0a'),
 'confidence': 100,
 'latitude': -37.446,
 'longitude': 148.102,
 'surface_temperature_celcius': 65}
{'_id': ObjectId('6290c48da64bd561823fff2a'),
 'confidence': 100,
 'latitude': -37.396,
 'longitude': 148.086,
 'surface_temperature_celcius': 65}
{'_id': ObjectId('6290c48da64bd561823fff59'),
 'confidence': 89,
 'latitude': -38.3998

 'surface_temperature_celcius': 73}
{'_id': ObjectId('6290c48da64bd561824003ff'),
 'confidence': 93,
 'latitude': -36.91,
 'longitude': 141.0704,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('6290c48da64bd56182400476'),
 'confidence': 93,
 'latitude': -36.8238,
 'longitude': 143.5724,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('6290c48da64bd561824004ae'),
 'confidence': 93,
 'latitude': -35.0616,
 'longitude': 141.4417,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('6290c48da64bd561824005ea'),
 'confidence': 94,
 'latitude': -36.3313,
 'longitude': 141.0017,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('6290c48da64bd5618240063d'),
 'confidence': 93,
 'latitude': -36.4489,
 'longitude': 144.7685,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('6290c48da64bd5618240080b'),
 'confidence': 93,
 'latitude': -36.8099,
 'longitude': 142.72799999999998,
 'surface_temperature_celcius': 73}
{'_id': ObjectId('6290c48da64bd56182400842'),
 'confidence': 71,
 'l

 'confidence': 98,
 'latitude': -37.7509,
 'longitude': 143.4023,
 'surface_temperature_celcius': 82}
{'_id': ObjectId('6290c48da64bd56182400813'),
 'confidence': 98,
 'latitude': -34.3526,
 'longitude': 141.6458,
 'surface_temperature_celcius': 82}
{'_id': ObjectId('6290c48da64bd561823ffe6e'),
 'confidence': 98,
 'latitude': -37.605,
 'longitude': 149.30200000000002,
 'surface_temperature_celcius': 83}
{'_id': ObjectId('6290c48da64bd561824000c5'),
 'confidence': 98,
 'latitude': -38.2853,
 'longitude': 145.9519,
 'surface_temperature_celcius': 83}
{'_id': ObjectId('6290c48da64bd5618240066a'),
 'confidence': 98,
 'latitude': -35.2381,
 'longitude': 142.994,
 'surface_temperature_celcius': 83}
{'_id': ObjectId('6290c48da64bd56182400724'),
 'confidence': 93,
 'latitude': -37.6439,
 'longitude': 142.91299999999998,
 'surface_temperature_celcius': 83}
{'_id': ObjectId('6290c48da64bd56182400866'),
 'confidence': 98,
 'latitude': -37.4261,
 'longitude': 147.0323,
 'surface_temperature_celciu

##### c. Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2021

In [17]:
# join the two datasets to show fields from both data
# use match and or to find the two desired dates, and project only necessary information
result = db.climate.aggregate([
    {
    "$lookup":{
        "from": "hotspot",
        "localField": "date",
        "foreignField": "date",
        "as":"hotspots",
        }
    },
    {
    "$match" : {"$or":[{"date": datetime(2021, 12, 15)},{"date":datetime(2021, 12, 16)}]}
    },
    {
    "$project" : {"date":1,"air_temperature_celcius":1,"relative_humidity":1,"max_wind_speed":1,"hotspots":{"surface_temperature_celcius":1}}  
    }
    ])
for x in result:
    pprint(x)
# pprint([x for x in result])

{'_id': ObjectId('6290c48ca64bd561823ffe23'),
 'air_temperature_celcius': 18,
 'date': datetime.datetime(2021, 12, 15, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 42},
              {'surface_temperature_celcius': 36},
              {'surface_temperature_celcius': 38},
              {'surface_temperature_celcius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'_id': ObjectId('6290c48ca64bd561823ffe24'),
 'air_temperature_celcius': 18,
 'date': datetime.datetime(2021, 12, 16, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 43},
              {'surface_temperature_celcius': 33},
              {'surface_temperature_celcius': 54},
              {'surface_temperature_celcius': 73},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 75},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 66},
              {'surface_temperature_celcius': 56},
              {'surface_tempera

##### d. Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.


In [18]:
# join based on date, then match the confidence between 80 and 100
# project necessary fields
result = db.hotspot.aggregate([{
    "$lookup":
    {
        "from": "climate",
        "localField": "date",
        "foreignField": "date",
        "as":"climate",
    }},
    {
    "$match" : {"confidence":{"$gte": 80, "$lte": 100}}
    },
    {
    "$project" : {"datetime": 1,
                "surface_temperature_celcius":1,"confidence":1,
                "climate":{"air_temperature_celcius":1}}  
    }
    ])
for x in result:
    pprint(x)
# pprint([x for x in result])

{'_id': ObjectId('6290c48da64bd561823ffe36'),
 'climate': [{'air_temperature_celcius': 28}],
 'confidence': 82,
 'datetime': datetime.datetime(2021, 12, 27, 0, 2, 15),
 'surface_temperature_celcius': 63}
{'_id': ObjectId('6290c48da64bd561823ffe38'),
 'climate': [{'air_temperature_celcius': 28}],
 'confidence': 86,
 'datetime': datetime.datetime(2021, 12, 27, 0, 2, 14),
 'surface_temperature_celcius': 67}
{'_id': ObjectId('6290c48da64bd561823ffe39'),
 'climate': [{'air_temperature_celcius': 17}],
 'confidence': 80,
 'datetime': datetime.datetime(2021, 12, 25, 4, 29, 8),
 'surface_temperature_celcius': 54}
{'_id': ObjectId('6290c48da64bd561823ffe3c'),
 'climate': [{'air_temperature_celcius': 18}],
 'confidence': 94,
 'datetime': datetime.datetime(2021, 12, 16, 15, 38, 39),
 'surface_temperature_celcius': 43}
{'_id': ObjectId('6290c48da64bd561823ffe3f'),
 'climate': [{'air_temperature_celcius': 18}],
 'confidence': 93,
 'datetime': datetime.datetime(2021, 12, 16, 4, 35, 13),
 'surface_tem

{'_id': ObjectId('6290c48da64bd561823fff0a'),
 'climate': [{'air_temperature_celcius': 14}],
 'confidence': 100,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 47),
 'surface_temperature_celcius': 65}
{'_id': ObjectId('6290c48da64bd561823fff0d'),
 'climate': [{'air_temperature_celcius': 14}],
 'confidence': 90,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 47),
 'surface_temperature_celcius': 41}
{'_id': ObjectId('6290c48da64bd561823fff0e'),
 'climate': [{'air_temperature_celcius': 14}],
 'confidence': 99,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 47),
 'surface_temperature_celcius': 61}
{'_id': ObjectId('6290c48da64bd561823fff11'),
 'climate': [{'air_temperature_celcius': 14}],
 'confidence': 94,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 45),
 'surface_temperature_celcius': 43}
{'_id': ObjectId('6290c48da64bd561823fff12'),
 'climate': [{'air_temperature_celcius': 14}],
 'confidence': 100,
 'datetime': datetime.datetime(2021, 9, 24, 15, 7, 45),
 'surface_temp

 'confidence': 100,
 'datetime': datetime.datetime(2021, 5, 4, 4, 48, 10),
 'surface_temperature_celcius': 89}
{'_id': ObjectId('6290c48da64bd56182400184'),
 'climate': [{'air_temperature_celcius': 10}],
 'confidence': 87,
 'datetime': datetime.datetime(2021, 5, 4, 4, 46, 50),
 'surface_temperature_celcius': 63}
{'_id': ObjectId('6290c48da64bd56182400186'),
 'climate': [{'air_temperature_celcius': 10}],
 'confidence': 81,
 'datetime': datetime.datetime(2021, 5, 4, 4, 46, 30),
 'surface_temperature_celcius': 54}
{'_id': ObjectId('6290c48da64bd56182400188'),
 'climate': [{'air_temperature_celcius': 10}],
 'confidence': 91,
 'datetime': datetime.datetime(2021, 5, 4, 4, 46, 20),
 'surface_temperature_celcius': 68}
{'_id': ObjectId('6290c48da64bd5618240018a'),
 'climate': [{'air_temperature_celcius': 10}],
 'confidence': 84,
 'datetime': datetime.datetime(2021, 5, 4, 4, 46, 20),
 'surface_temperature_celcius': 58}
{'_id': ObjectId('6290c48da64bd5618240018b'),
 'climate': [{'air_temperature_

{'_id': ObjectId('6290c48da64bd56182400310'),
 'climate': [{'air_temperature_celcius': 15}],
 'confidence': 80,
 'datetime': datetime.datetime(2021, 4, 18, 4, 46, 30),
 'surface_temperature_celcius': 53}
{'_id': ObjectId('6290c48da64bd56182400314'),
 'climate': [{'air_temperature_celcius': 15}],
 'confidence': 90,
 'datetime': datetime.datetime(2021, 4, 18, 4, 46, 30),
 'surface_temperature_celcius': 67}
{'_id': ObjectId('6290c48da64bd56182400319'),
 'climate': [{'air_temperature_celcius': 15}],
 'confidence': 93,
 'datetime': datetime.datetime(2021, 4, 18, 4, 46),
 'surface_temperature_celcius': 73}
{'_id': ObjectId('6290c48da64bd5618240031b'),
 'climate': [{'air_temperature_celcius': 15}],
 'confidence': 95,
 'datetime': datetime.datetime(2021, 4, 18, 4, 45, 50),
 'surface_temperature_celcius': 77}
{'_id': ObjectId('6290c48da64bd5618240031c'),
 'climate': [{'air_temperature_celcius': 15}],
 'confidence': 97,
 'datetime': datetime.datetime(2021, 4, 18, 4, 45, 50),
 'surface_temperatur

 'confidence': 80,
 'datetime': datetime.datetime(2021, 4, 15, 4, 14, 20),
 'surface_temperature_celcius': 53}
{'_id': ObjectId('6290c48da64bd56182400488'),
 'climate': [{'air_temperature_celcius': 12}],
 'confidence': 80,
 'datetime': datetime.datetime(2021, 4, 15, 4, 14, 20),
 'surface_temperature_celcius': 54}
{'_id': ObjectId('6290c48da64bd56182400489'),
 'climate': [{'air_temperature_celcius': 12}],
 'confidence': 100,
 'datetime': datetime.datetime(2021, 4, 15, 4, 14, 20),
 'surface_temperature_celcius': 100}
{'_id': ObjectId('6290c48da64bd5618240048c'),
 'climate': [{'air_temperature_celcius': 13}],
 'confidence': 82,
 'datetime': datetime.datetime(2021, 4, 14, 5, 15, 50),
 'surface_temperature_celcius': 55}
{'_id': ObjectId('6290c48da64bd56182400490'),
 'climate': [{'air_temperature_celcius': 13}],
 'confidence': 87,
 'datetime': datetime.datetime(2021, 4, 14, 5, 9, 10),
 'surface_temperature_celcius': 62}
{'_id': ObjectId('6290c48da64bd56182400492'),
 'climate': [{'air_tempera

 'surface_temperature_celcius': 93}
{'_id': ObjectId('6290c48da64bd561824005da'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence': 95,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_celcius': 77}
{'_id': ObjectId('6290c48da64bd561824005dc'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence': 81,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_celcius': 54}
{'_id': ObjectId('6290c48da64bd561824005dd'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence': 95,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_celcius': 75}
{'_id': ObjectId('6290c48da64bd561824005de'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence': 84,
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'surface_temperature_celcius': 58}
{'_id': ObjectId('6290c48da64bd561824005df'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence': 84,
 'datetime': datetime.datetime(2021,

 'datetime': datetime.datetime(2021, 4, 4, 4, 33, 10),
 'surface_temperature_celcius': 62}
{'_id': ObjectId('6290c48da64bd56182400777'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence': 82,
 'datetime': datetime.datetime(2021, 4, 4, 4, 33, 10),
 'surface_temperature_celcius': 62}
{'_id': ObjectId('6290c48da64bd5618240077a'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence': 97,
 'datetime': datetime.datetime(2021, 4, 4, 4, 33),
 'surface_temperature_celcius': 79}
{'_id': ObjectId('6290c48da64bd5618240077b'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence': 86,
 'datetime': datetime.datetime(2021, 4, 4, 4, 33),
 'surface_temperature_celcius': 61}
{'_id': ObjectId('6290c48da64bd5618240077c'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence': 98,
 'datetime': datetime.datetime(2021, 4, 4, 4, 32, 50),
 'surface_temperature_celcius': 82}
{'_id': ObjectId('6290c48da64bd5618240077d'),
 'climate': [{'air_temperature_celcius': 16}],
 'confidence

##### e. Find the top 10 records with the highest surface temperature (°C)

In [19]:
# sort in descending order, and only show 10 records
results = hotspot_collection.aggregate([{"$sort": {"surface_temperature_celcius":-1}},{"$limit": 10}])
for x in results:
    pprint(x)

{'_id': ObjectId('6290c48da64bd561824002d8'),
 'confidence': 100,
 'date': datetime.datetime(2021, 4, 18, 0, 0),
 'datetime': datetime.datetime(2021, 4, 18, 4, 52),
 'latitude': -38.1665,
 'longitude': 143.062,
 'surface_temperature_celcius': 124}
{'_id': ObjectId('6290c48da64bd56182400782'),
 'confidence': 100,
 'date': datetime.datetime(2021, 4, 4, 0, 0),
 'datetime': datetime.datetime(2021, 4, 4, 4, 32, 50),
 'latitude': -36.343,
 'longitude': 142.1986,
 'surface_temperature_celcius': 123}
{'_id': ObjectId('6290c48da64bd5618240024a'),
 'confidence': 100,
 'date': datetime.datetime(2021, 5, 1, 0, 0),
 'datetime': datetime.datetime(2021, 5, 1, 4, 14, 20),
 'latitude': -36.9318,
 'longitude': 143.0907,
 'surface_temperature_celcius': 122}
{'_id': ObjectId('6290c48da64bd56182400871'),
 'confidence': 100,
 'date': datetime.datetime(2021, 3, 18, 0, 0),
 'datetime': datetime.datetime(2021, 3, 18, 3, 50, 50),
 'latitude': -37.016999999999996,
 'longitude': 148.1297,
 'surface_temperature_ce

##### f. Find the number of fires each day. You are required to only display the total number of fires and the date in the output.


In [20]:
# join based on date, and only project necessary fields based on the length of the hotspots list
result = db.climate.aggregate([
    {
    "$lookup":{
        "from": "hotspot",
        "localField": "date",
        "foreignField": "date",
        "as":"Number of fires",
        }
    },
    {"$project": {"date":1,"Number of fires":{"$size":"$Number of fires"},"_id":0}}
])
for x in result:
    pprint(x)

{'Number of fires': 0, 'date': datetime.datetime(2020, 12, 31, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 2, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 3, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 4, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 5, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 6, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 7, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 8, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 9, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 10, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 11, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 12, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 13, 0, 0)}
{'Number of fires': 0, 'date': datetime.datetime(2021, 1, 14, 0, 0)}
{'Number of fires': 0, 'date': datetime.d

##### g. Find the records of fires where the confidence is below 70.



In [21]:
result = db.hotspot.aggregate([{
    "$lookup":
    {
        "from": "climate",
        "localField": "date",
        "foreignField": "date",
        "as":"climate",
    }},
    {
    "$match" : {"confidence":{"$lt": 70}}
    }])
for x in result:
    pprint(x)

{'_id': ObjectId('6290c48da64bd561823ffe37'),
 'climate': [{'GHI_w/m2': 223,
              '_id': ObjectId('6290c48ca64bd561823ffe2f'),
              'air_temperature_celcius': 28,
              'date': datetime.datetime(2021, 12, 27, 0, 0),
              'max_wind_speed': 15.9,
              'precipitation ': 0.0,
              'precipitation_type': 'I',
              'relative_humidity': 58.3,
              'station': 948702,
              'windspeed_knots': 9.3}],
 'confidence': 67,
 'date': datetime.datetime(2021, 12, 27, 0, 0),
 'datetime': datetime.datetime(2021, 12, 27, 0, 2, 15),
 'latitude': -35.554,
 'longitude': 143.30700000000002,
 'surface_temperature_celcius': 53}
{'_id': ObjectId('6290c48da64bd561823ffe3a'),
 'climate': [{'GHI_w/m2': 141,
              '_id': ObjectId('6290c48ca64bd561823ffe2c'),
              'air_temperature_celcius': 16,
              'date': datetime.datetime(2021, 12, 24, 0, 0),
              'max_wind_speed': 16.9,
              'precipitation ': 0

{'_id': ObjectId('6290c48da64bd561823fff1b'),
 'climate': [{'GHI_w/m2': 126,
              '_id': ObjectId('6290c48ca64bd561823ffdd1'),
              'air_temperature_celcius': 14,
              'date': datetime.datetime(2021, 9, 24, 0, 0),
              'max_wind_speed': 23.9,
              'precipitation ': 0.16,
              'precipitation_type': 'G',
              'relative_humidity': 44.0,
              'station': 948702,
              'windspeed_knots': 14.4}],
 'confidence': 68,
 'date': datetime.datetime(2021, 9, 24, 0, 0),
 'datetime': datetime.datetime(2021, 9, 24, 4, 4, 18),
 'latitude': -37.452,
 'longitude': 148.115,
 'surface_temperature_celcius': 77}
{'_id': ObjectId('6290c48da64bd561823fff1c'),
 'climate': [{'GHI_w/m2': 126,
              '_id': ObjectId('6290c48ca64bd561823ffdd1'),
              'air_temperature_celcius': 14,
              'date': datetime.datetime(2021, 9, 24, 0, 0),
              'max_wind_speed': 23.9,
              'precipitation ': 0.16,
        

{'_id': ObjectId('6290c48da64bd561823fff80'),
 'climate': [{'GHI_w/m2': 87,
              '_id': ObjectId('6290c48ca64bd561823ffd6f'),
              'air_temperature_celcius': 10,
              'date': datetime.datetime(2021, 6, 18, 0, 0),
              'max_wind_speed': 7.0,
              'precipitation ': 0.0,
              'precipitation_type': 'I',
              'relative_humidity': 47.9,
              'station': 948701,
              'windspeed_knots': 4.1}],
 'confidence': 53,
 'date': datetime.datetime(2021, 6, 18, 0, 0),
 'datetime': datetime.datetime(2021, 6, 18, 4, 14),
 'latitude': -37.8387,
 'longitude': 144.6301,
 'surface_temperature_celcius': 39}
{'_id': ObjectId('6290c48da64bd561823fff81'),
 'climate': [{'GHI_w/m2': 73,
              '_id': ObjectId('6290c48ca64bd561823ffd6d'),
              'air_temperature_celcius': 8,
              'date': datetime.datetime(2021, 6, 16, 0, 0),
              'max_wind_speed': 6.0,
              'precipitation ': 0.0,
              'pr

              'relative_humidity': 47.0,
              'station': 948701,
              'windspeed_knots': 5.2}],
 'confidence': 63,
 'date': datetime.datetime(2021, 5, 15, 0, 0),
 'datetime': datetime.datetime(2021, 5, 15, 4, 27, 10),
 'latitude': -36.3542,
 'longitude': 141.0935,
 'surface_temperature_celcius': 41}
{'_id': ObjectId('6290c48da64bd56182400016'),
 'climate': [{'GHI_w/m2': 88,
              '_id': ObjectId('6290c48ca64bd561823ffd4d'),
              'air_temperature_celcius': 10,
              'date': datetime.datetime(2021, 5, 15, 0, 0),
              'max_wind_speed': 8.9,
              'precipitation ': 0.0,
              'precipitation_type': 'I',
              'relative_humidity': 47.0,
              'station': 948701,
              'windspeed_knots': 5.2}],
 'confidence': 57,
 'date': datetime.datetime(2021, 5, 15, 0, 0),
 'datetime': datetime.datetime(2021, 5, 15, 4, 26, 40),
 'latitude': -35.6785,
 'longitude': 143.5077,
 'surface_temperature_celcius': 39}
{'_id':

              'windspeed_knots': 2.8}],
 'confidence': 59,
 'date': datetime.datetime(2021, 5, 13, 0, 0),
 'datetime': datetime.datetime(2021, 5, 13, 4, 38, 40),
 'latitude': -36.304,
 'longitude': 143.7445,
 'surface_temperature_celcius': 40}
{'_id': ObjectId('6290c48da64bd56182400081'),
 'climate': [{'GHI_w/m2': 81,
              '_id': ObjectId('6290c48ca64bd561823ffd4b'),
              'air_temperature_celcius': 9,
              'date': datetime.datetime(2021, 5, 13, 0, 0),
              'max_wind_speed': 8.0,
              'precipitation ': 0.0,
              'precipitation_type': 'I',
              'relative_humidity': 44.0,
              'station': 948701,
              'windspeed_knots': 2.8}],
 'confidence': 55,
 'date': datetime.datetime(2021, 5, 13, 0, 0),
 'datetime': datetime.datetime(2021, 5, 13, 4, 38, 40),
 'latitude': -36.591,
 'longitude': 141.7194,
 'surface_temperature_celcius': 40}
{'_id': ObjectId('6290c48da64bd56182400082'),
 'climate': [{'GHI_w/m2': 81,
        

...

 'climate': [{'GHI_w/m2': 132,
              '_id': ObjectId('6290c48ca64bd561823ffd2d'),
              'air_temperature_celcius': 16,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'max_wind_speed': 9.9,
              'precipitation ': 0.0,
              'precipitation_type': 'I',
              'relative_humidity': 54.9,
              'station': 948701,
              'windspeed_knots': 5.9}],
 'confidence': 58,
 'date': datetime.datetime(2021, 4, 13, 0, 0),
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 40),
 'latitude': -38.26,
 'longitude': 143.9602,
 'surface_temperature_celcius': 40}
{'_id': ObjectId('6290c48da64bd5618240053d'),
 'climate': [{'GHI_w/m2': 132,
              '_id': ObjectId('6290c48ca64bd561823ffd2d'),
              'air_temperature_celcius': 16,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'max_wind_speed': 9.9,
              'precipitation ': 0.0,
              'precipitation_type': 'I',
              're

 'confidence': 62,
 'date': datetime.datetime(2021, 4, 13, 0, 0),
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'latitude': -38.2583,
 'longitude': 143.9717,
 'surface_temperature_celcius': 41}
{'_id': ObjectId('6290c48da64bd561824005f9'),
 'climate': [{'GHI_w/m2': 132,
              '_id': ObjectId('6290c48ca64bd561823ffd2d'),
              'air_temperature_celcius': 16,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'max_wind_speed': 9.9,
              'precipitation ': 0.0,
              'precipitation_type': 'I',
              'relative_humidity': 54.9,
              'station': 948701,
              'windspeed_knots': 5.9}],
 'confidence': 65,
 'date': datetime.datetime(2021, 4, 13, 0, 0),
 'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
 'latitude': -36.6843,
 'longitude': 141.5899,
 'surface_temperature_celcius': 43}
{'_id': ObjectId('6290c48da64bd561824005fb'),
 'climate': [{'GHI_w/m2': 132,
              '_id': ObjectId('6290c48ca64b

 'climate': [{'GHI_w/m2': 159,
              '_id': ObjectId('6290c48ca64bd561823ffd28'),
              'air_temperature_celcius': 19,
              'date': datetime.datetime(2021, 4, 8, 0, 0),
              'max_wind_speed': 21.0,
              'precipitation ': 0.47,
              'precipitation_type': 'G',
              'relative_humidity': 52.8,
              'station': 948701,
              'windspeed_knots': 12.5}],
 'confidence': 66,
 'date': datetime.datetime(2021, 4, 8, 0, 0),
 'datetime': datetime.datetime(2021, 4, 8, 4, 8, 40),
 'latitude': -37.3493,
 'longitude': 149.3691,
 'surface_temperature_celcius': 80}
{'_id': ObjectId('6290c48da64bd5618240066d'),
 'climate': [{'GHI_w/m2': 159,
              '_id': ObjectId('6290c48ca64bd561823ffd28'),
              'air_temperature_celcius': 19,
              'date': datetime.datetime(2021, 4, 8, 0, 0),
              'max_wind_speed': 21.0,
              'precipitation ': 0.47,
              'precipitation_type': 'G',
              '

              'precipitation ': 0.0,
              'precipitation_type': 'I',
              'relative_humidity': 47.2,
              'station': 948701,
              'windspeed_knots': 4.9}],
 'confidence': 66,
 'date': datetime.datetime(2021, 4, 5, 0, 0),
 'datetime': datetime.datetime(2021, 4, 5, 5, 15, 40),
 'latitude': -37.7522,
 'longitude': 142.3183,
 'surface_temperature_celcius': 50}
{'_id': ObjectId('6290c48da64bd56182400723'),
 'climate': [{'GHI_w/m2': 149,
              '_id': ObjectId('6290c48ca64bd561823ffd25'),
              'air_temperature_celcius': 17,
              'date': datetime.datetime(2021, 4, 5, 0, 0),
              'max_wind_speed': 13.0,
              'precipitation ': 0.0,
              'precipitation_type': 'I',
              'relative_humidity': 47.2,
              'station': 948701,
              'windspeed_knots': 4.9}],
 'confidence': 66,
 'date': datetime.datetime(2021, 4, 5, 0, 0),
 'datetime': datetime.datetime(2021, 4, 5, 5, 15, 30),
 'latitude': -3

 'climate': [{'GHI_w/m2': 124,
              '_id': ObjectId('6290c48ca64bd561823ffd20'),
              'air_temperature_celcius': 14,
              'date': datetime.datetime(2021, 3, 31, 0, 0),
              'max_wind_speed': 9.9,
              'precipitation ': 0.0,
              'precipitation_type': 'G',
              'relative_humidity': 45.8,
              'station': 948701,
              'windspeed_knots': 6.5}],
 'confidence': 68,
 'date': datetime.datetime(2021, 3, 31, 0, 0),
 'datetime': datetime.datetime(2021, 3, 31, 4, 57, 50),
 'latitude': -36.1649,
 'longitude': 145.9352,
 'surface_temperature_celcius': 44}
{'_id': ObjectId('6290c48da64bd561824007f8'),
 'climate': [{'GHI_w/m2': 124,
              '_id': ObjectId('6290c48ca64bd561823ffd20'),
              'air_temperature_celcius': 14,
              'date': datetime.datetime(2021, 3, 31, 0, 0),
              'max_wind_speed': 9.9,
              'precipitation ': 0.0,
              'precipitation_type': 'G',
              '

##### h. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.



In [22]:
result = db.climate.aggregate([
    {
    "$lookup":{
        "from": "hotspot",
        "localField": "date",
        "foreignField": "date",
        "as":"hotspot",
        }
    },
    {"$project": {"_id":0,"date":1,"average_surface_temp":{"$avg":"$hotspot.surface_temperature_celcius"}}}
])
    
for x in result:
    pprint(x)


{'average_surface_temp': None, 'date': datetime.datetime(2020, 12, 31, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 2, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 3, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 4, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 5, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 6, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 7, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 8, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 9, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 10, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 11, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 12, 0, 0)}
{'average_surface_temp': None, 'date': datetime.datetime(2021, 1, 13, 0, 0)}
{'aver

##### i. Find the top 10 records with the lowest GHI.


In [23]:
results = climate_collection.aggregate([
    {"$limit": 10},
    {"$sort": {"GHI_w/m2":1}},
    {"$lookup":
    {
        "from": "hotspot",
        "localField": "date",
        "foreignField": "date",
        "as":"hotspot",
    }}])
for x in results:
    pprint(x)

{'GHI_w/m2': 128,
 '_id': ObjectId('6290c48ca64bd561823ffcc8'),
 'air_temperature_celcius': 15,
 'date': datetime.datetime(2021, 1, 2, 0, 0),
 'hotspot': [],
 'max_wind_speed': 13.0,
 'precipitation ': 0.02,
 'precipitation_type': 'G',
 'relative_humidity': 50.7,
 'station': 948700,
 'windspeed_knots': 9.2}
{'GHI_w/m2': 133,
 '_id': ObjectId('6290c48ca64bd561823ffcc9'),
 'air_temperature_celcius': 16,
 'date': datetime.datetime(2021, 1, 3, 0, 0),
 'hotspot': [],
 'max_wind_speed': 15.0,
 'precipitation ': 0.0,
 'precipitation_type': 'G',
 'relative_humidity': 53.6,
 'station': 948700,
 'windspeed_knots': 8.1}
{'GHI_w/m2': 154,
 '_id': ObjectId('6290c48ca64bd561823ffcc7'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2020, 12, 31, 0, 0),
 'hotspot': [],
 'max_wind_speed': 11.1,
 'precipitation ': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}
{'GHI_w/m2': 154,
 '_id': ObjectId('6290c48ca64bd561823ffccf'),
 'air_tem

##### j. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.


In [24]:
result = db.climate.aggregate([{
    "$lookup":
    {
        "from": "hotspot",
        "localField": "date",
        "foreignField": "date",
        "as":"hotspot",
    }},
    {
    "$match" : {"precipitation_type": "G", "precipitation ": {"$gte": 0.20, "$lte": 0.35}}
    }])
for x in result:
    pprint(x)
# pprint([x for x in result])

{'GHI_w/m2': 157,
 '_id': ObjectId('6290c48ca64bd561823ffcd3'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2021, 1, 13, 0, 0),
 'hotspot': [],
 'max_wind_speed': 18.1,
 'precipitation ': 0.31,
 'precipitation_type': 'G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 '_id': ObjectId('6290c48ca64bd561823ffd1e'),
 'air_temperature_celcius': 17,
 'date': datetime.datetime(2021, 3, 29, 0, 0),
 'hotspot': [{'_id': ObjectId('6290c48da64bd56182400804'),
              'confidence': 69,
              'date': datetime.datetime(2021, 3, 29, 0, 0),
              'datetime': datetime.datetime(2021, 3, 29, 0, 48, 40),
              'latitude': -34.2648,
              'longitude': 141.6325,
              'surface_temperature_celcius': 51}],
 'max_wind_speed': 21.0,
 'precipitation ': 0.24,
 'precipitation_type': 'G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 '_id': ObjectId('6290c48ca64bd5